In [ ]:
!pip install text2graphapi

In [ ]:
import pandas as pd
data = pd.read_csv("en_subtask1.csv")
data.head()

#recortamos la data para las pruebas
data_recortada = data.head(60)
dataprueba = 'dataprueba.csv'
data_recortada.to_csv(dataprueba, index=False)

data_human = data_recortada.loc[data_recortada['label'] == 'human']
data_generated = data_recortada.loc[data_recortada['label'] == 'generated']

data_human['filas'] = range(1, len(data_human) + 1)
data_generated['filas'] = range(1,len(data_generated)+1)

In [ ]:
tex_to_graph_human = []
for index, row in data_human.iterrows():
    fila = {"id": row['filas'], "doc": row['text']}
    tex_to_graph_human.append(fila)

In [ ]:
tex_to_graph_generated = []

for index, row in data_generated.iterrows():
    fila = {"id": row['filas'], "doc": row['text']}
    tex_to_graph_generated.append(fila)


In [ ]:
from text2graphapi.src.Cooccurrence import Cooccurrence

to_word_coocc_graph = Cooccurrence(graph_type = 'DiGraph',
        language = 'en', apply_prep = True,
        window_size = 3, output_format = 'networkx')


In [ ]:
Grafos_cooncurencia_human = to_word_coocc_graph.transform(tex_to_graph_human)
Grafos_cooncurencia_generated = to_word_coocc_graph.transform(tex_to_graph_generated)

In [ ]:
lista_grafos_humanos = []

for i in Grafos_cooncurencia_human:
  grafo = i['graph']
  lista_grafos_humanos.append(grafo)

lista_grafos_humanos

In [ ]:
lista_grafos_generated = []
for i in Grafos_cooncurencia_generated:
  grafo = i['graph']
  lista_grafos_generated.append(grafo)

lista_grafos_generated

In [ ]:
#eficiencia global de conectividad en humanos
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import approximation as approx



lista1 = []
for grafo in lista_grafos_humanos:
  grafo = grafo.to_undirected()
  lista1.append(nx.global_efficiency(grafo))

  print(nx.global_efficiency(grafo))

print("La eficiencia global en promedio es:", sum(lista1)/len(lista1))



In [ ]:
#eficiencia global de conectividad en IA's
lista2 = []
for grafo in lista_grafos_generated:
  grafo = grafo.to_undirected()
  lista2.append(nx.global_efficiency(grafo))

  print(nx.global_efficiency(grafo))

print("La eficiencia global en promedio es:", sum(lista2)/len(lista2))



In [ ]:
IAefg = sum(lista2)/len(lista2)
Hefg = sum(lista1)/len(lista1)
diferencia = abs(IAefg - Hefg)
print("la diferencia de eficiencias globales es:",diferencia)

In [ ]:
#promedio de cuantos nodos están conectados con otros nodos en humanos
lista3 = []
for grafo in lista_grafos_humanos:
  lista3.append(nx.average_neighbor_degree(grafo))

print(lista3)

# extraer los valores de los diccionarios
valores_numericosH = [valor for diccionario in lista3 for valor in diccionario.values() if isinstance(valor, (int, float))]

print(valores_numericosH)

average_neighbor_degree_promedio = sum(valores_numericosH)/len(valores_numericosH)
print("El promedio de cuantos nodos están conectados con otros nodos en humanos es:", average_neighbor_degree_promedio)

In [ ]:
#Promedio de cuantos nodos están conectados con otros nodos en IA
lista4 = []

for grafo in lista_grafos_generated:
  lista4.append(nx.average_neighbor_degree(grafo))

print(lista4)

valores_numericosIA = [valor for diccionario in lista4 for valor in diccionario.values() if isinstance(valor, (int, float))]

print(valores_numericosIA)
average_neighbor_degree_promedioIA = sum(valores_numericosIA)/len(valores_numericosIA)
print("El promedio de cuantos nodos están conectados con otros nodos en IA es:", average_neighbor_degree_promedioIA)

In [ ]:
#Diferecia de promedios
print("La diferencia de promedios de nodos conectados es:",
      abs(average_neighbor_degree_promedio - average_neighbor_degree_promedioIA))

In [ ]:
#veamos ahora la centralidad de cercania en humanos
lista5 = []
for grafo in lista_grafos_humanos:
  lista5.append(nx.closeness_centrality(grafo))

print(lista5)
valoresH = [valor for diccionario in lista5 for valor in diccionario.values() if isinstance(valor, (int, float))]
print(valoresH)
promedio_cercaniaH = sum(valoresH)/len(valoresH)
print("El promedio de centralidad de cercania en humanos es:", promedio_cercaniaH)

In [ ]:
#los mismo pero en IA
lista6 = []
for grafo in lista_grafos_generated:
  lista6.append(nx.closeness_centrality(grafo))

print(lista6)
valoresIA = [valor for diccionario in lista6 for valor in diccionario.values() if isinstance(valor, (int, float))]
print(valoresIA)
promedio_cercaniaIA = sum(valoresIA)/len(valoresIA)
print("El promedio de centralidad de cercania en humanos es:", promedio_cercaniaIA)

In [ ]:
print("La diferencia de cercania de centralidad es:",
      abs(promedio_cercaniaIA - promedio_cercaniaH))

In [ ]:
#conjunto dominante ia
lista9 = []
for i in lista_grafos_generated:
  lista9.append(nx.dominating_set(i, start_with=None))

listapalabrasIA = []
for conjunto in lista9:
  listapalabrasIA.extend(conjunto)
print(listapalabrasIA)



In [ ]:

#conjunto dominante humanos
lista7 = []
for i in lista_grafos_humanos:
  lista7.append(nx.dominating_set(i, start_with=None))

listaconpalabrashumano = []
for conjunto in lista7:
  listaconpalabrashumano.extend(conjunto)

print(listaconpalabrashumano)

In [ ]:
print("HUMANO:", sorted(listaconpalabrashumano))
print("IA:", sorted(listapalabrasIA))

In [ ]:
textoIA = ' '.join(listapalabrasIA)

# Crear un objeto WordCloud
nube_palabrasIA = WordCloud(width=800, height=400, background_color='white').generate(textoIA)

# Mostrar la nube de palabras utilizando matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(nube_palabrasIA, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
textoHumano = ' '.join(listaconpalabrashumano)

# Crear un objeto WordCloud
nube_palabras = WordCloud(width=800, height=400, background_color='white').generate(textoHumano)

# Mostrar la nube de palabras utilizando matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(nube_palabras, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
pip install wordcloud matplotlib